# ASN.1 Coding Examples


Als erster Schritt werden die asn1tools im Python Kernel installiert.


In [1]:
# Install a conda package in the current Jupyter kernel
import sys
!pip install asn1tools-0.146.6-py2.py3-none-any.whl

Processing ./asn1tools-0.146.6-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ee/cc/d992e1d886d5ce15d2622c2e89b6de52b48312c6f05e34b7ee881b4ccb02/diskcache-4.1.0-py2.py3-none-any.whl
Processing ./.cache/pip/wheels/5b/c3/e5/27b8d419b20cd587df1ccf36cd7f7ca79486185e87085cec13/bitstruct-8.9.0-cp37-cp37m-linux_x86_64.whl


In [36]:
import sys
!pip install pixiedust

Processing ./.cache/pip/wheels/e8/b1/86/c2f2e16e6bf9bfe556f9dbf8adb9f41816c476d73078c7d0eb/pixiedust-1.1.18-cp37-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e4/8d/9e28e9af95739e6d2d2f8d4bef0b3432da40b7c3588fbad4298c1be09e48/geojson-2.5.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/2b/03/13dde6512ad7b4557eb792fbcf0c653af6076b81e5941d36ec61f7ce6028/astunparse-1.6.3-py2.py3-none-any.whl
Processing ./.cache/pip/wheels/c0/47/fb/8a64f89aecfe0059830479308ad42d62e898a3e3cefdf6ba28/mpld3-0.3-cp37-none-any.whl
  Using cached https://files.pythonhosted.org/packages/74/46/e81907704ab203206769dee1385dc77e1407576ff8f50a0681d0a6b541be/colour-0.1.5-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ab/c4/ba46d44855e6eb1770a12edace5a165a0c6de13349f592b9036257f3c3d3/Markdown-3.2.1-py2.py3-none-any.whl


In [37]:
import pixiedust

Pixiedust database opened successfully


In [3]:
import binascii
bytes = b'\xf0\x0f'

def print_bytes_as_bits(bytes) :
    return ''.join("{:08b}\n".format(byte) for byte in bytes)

def hexlify(bytes) :
    return binascii.hexlify(bytearray(bytes))

print("Darstellung als Binary Code:")
print (print_bytes_as_bits(bytes) )
print("Darstellung als Hex Code:")
print (hexlify (bytes) )

Darstellung als Binary Code:
11110000
00001111

Darstellung als Hex Code:
b'f00f'


## ASN1 Coding Examples

Interessant in Zusammenhang mit dem eCall ist nur das UPER Encoding Schema. 

Das folgende ASN.1 Schema definiert verschiedene Datentypen. 

Vergleiche die mit dem ASN.1 Onlnr EnCoder. 

[https://asn1.io/asn1playground](https://asn1.io/asn1playground)

%%html
<a href="https://asn1.io/asn1playground">https://asn1.io/asn1playground</a>







In [4]:
#%%pixie_debugger -b uper.encode

asn1_Schema = """
X DEFINITIONS
AUTOMATIC TAGS  ::= BEGIN

S1 ::= SEQUENCE {
a BOOLEAN,
b INTEGER(0 .. 255) OPTIONAL,
c IA5String OPTIONAL,
d OCTET STRING OPTIONAL
}

S2 ::= SEQUENCE {
a BOOLEAN,
b INTEGER OPTIONAL,
...,
...,
d OCTET STRING OPTIONAL
}

S ::= SEQUENCE {
   name    UTF8String,
   age     INTEGER
}

O ::= OCTET STRING (CONTAINING S)


VT ::= SEQUENCE {
gasolineTankPresent BOOLEAN DEFAULT FALSE,
dieselTankPresent BOOLEAN DEFAULT FALSE,
compressedNaturalGas BOOLEAN DEFAULT FALSE,
liquidPropaneGas BOOLEAN DEFAULT FALSE,
electricEnergyStorage BOOLEAN DEFAULT FALSE,
hydrogenStorage BOOLEAN DEFAULT FALSE,
otherStorage BOOLEAN DEFAULT FALSE,
...
}


END
"""

mess =  {
			'gasolineTankPresent' : True,
			'dieselTankPresent' : False,
			'compressedNaturalGas' : False,
			'liquidPropaneGas' : False,
			'electricEnergyStorage' : False,
			'hydrogenStorage' : False,
			'otherStorage' : False
}


import binascii
import asn1tools
uper = asn1tools.compile_string(asn1_Schema, codec='uper' )

gaso = uper.encode('VT', mess, check_types=True, check_constraints=True)


print(print_bytes_as_bits(gaso))
print(hexlify(gaso))


01000000
10000000

b'4080'


Test mit optionalen Feldern (Boolean default true/false)

In [2]:
import binascii
import asn1tools

foo = asn1tools.compile_files('data/optional.asn1', codec='xer')
uper = asn1tools.compile_files('data/optional.asn1', codec='uper' )

encoded = foo.encode('S1', {'a': True, 'b': 255, 'c': 'xx' , 'd' : b'\xff\x00\xff' })

uec = uper.encode('S1', {'a': True, 'b': 255, 'c': 'HHHH',  'd' : b'\xff\x00\xff' }, check_types=True, check_constraints=True)
uec = uper.encode('S1', {'a': True, 'b': 255, 'c': '@@@@',  'd' : b'\xff\x00\xff' }, check_types=True, check_constraints=True)
uec = uper.encode('S1', {'a': True, 'b': 255,   'd' : b'\xff\x00\xff' }, check_types=True, check_constraints=True)


print ("BER encoding TLV   nach X.690 ITU-T Standard")
print ("--------------------------------------------")
print(encoded)
print("".join("\\x%02x " % i for i in encoded))

print ("UPER encoding von S1 mit optionalen Felder nach X.690 ITU-T Standard")
print("".join("\\x%02x " % i for i in uec))
print("".join("{:08b}\n".format(i)  for i in uec))


print("---------------------------------------------")
print("Decoding ...")
result = uper.decode('S1', uec)
print(result)

#result = foo.decode('S2', encoded)
#print(result)

print (chr(64))

BER encoding TLV   nach X.690 ITU-T Standard
--------------------------------------------
b'<S1><a><true /></a><b>255</b><c>xx</c><d>FF00FF</d></S1>'
\x3c \x53 \x31 \x3e \x3c \x61 \x3e \x3c \x74 \x72 \x75 \x65 \x20 \x2f \x3e \x3c \x2f \x61 \x3e \x3c \x62 \x3e \x32 \x35 \x35 \x3c \x2f \x62 \x3e \x3c \x63 \x3e \x78 \x78 \x3c \x2f \x63 \x3e \x3c \x64 \x3e \x46 \x46 \x30 \x30 \x46 \x46 \x3c \x2f \x64 \x3e \x3c \x2f \x53 \x31 \x3e 
UPER encoding von S1 mit optionalen Felder nach X.690 ITU-T Standard
\xbf \xf0 \x3f \xf0 \x0f \xf0 
10111111
11110000
00111111
11110000
00001111
11110000

---------------------------------------------
Decoding ...
{'a': True, 'b': 255, 'd': b'\xff\x00\xff'}
@


Kommentar zu UPER Encoding.
Die ersten 3-bit geben die optionalen Parameter in S1 an. Im Beispiel ist z.B. der 'c' String nicht gesetzt. 
Danach folgt der 8Bit Integer Wert (255).
Danach IA5String (7Bit nach ISO T.50 Standard), Länge als 8Bit danach 7Bit Sequenzen. Falls der String leer ist, so ist die Länge 0.

Rest der Bytes mit 0 aufgefüllt.


In [6]:
print(uper.types['S1'])

Sequence(S1, [Boolean(a), Integer(b), IA5String(c), OctetString(d)])


Containing Keyword

In [24]:
import binascii
sEncoded = foo.encode('S', {'name': 'Calvin', 'age': 5})
print(sEncoded)
print("".join("\\x%02x " % i for i in sEncoded))
print(binascii.hexlify(sEncoded))

oEndcoded = foo.encode('O', b'0\x0b\x0c\x06Calvin\x02\x01\x05')
print(oEndcoded)
print(binascii.hexlify(oEndcoded))

b'<S><name>Calvin</name><age>5</age></S>'
\x3c \x53 \x3e \x3c \x6e \x61 \x6d \x65 \x3e \x43 \x61 \x6c \x76 \x69 \x6e \x3c \x2f \x6e \x61 \x6d \x65 \x3e \x3c \x61 \x67 \x65 \x3e \x35 \x3c \x2f \x61 \x67 \x65 \x3e \x3c \x2f \x53 \x3e 
b'3c533e3c6e616d653e43616c76696e3c2f6e616d653e3c6167653e353c2f6167653e3c2f533e'
b'<O>300B0C0643616C76696E020105</O>'
b'3c4f3e33303042304330363433363136433736363936453032303130353c2f4f3e'


# ASN.1 Coding

In [25]:

import asn1tools
foo = asn1tools.compile_files('data/foo.asn', codec='ber')
encoded = foo.encode('Question', {'id': 1, 'question': 'Is 1+1=3?'})
print(encoded)
encoded = foo.encode('Answer', {'id': 2, 'answer': False})
print(encoded)


bytearray(b'0\x0e\x02\x01\x01\x16\tIs 1+1=3?')
bytearray(b'0\x06\x02\x01\x02\x01\x01\x00')


Darstellung als Binary Code:
11110000
00001111

Darstellung als Hex Code:
b'f00f'


In [7]:
import binascii
binascii.hexlify(bytearray(encoded))

b'3006020102010100'

In [3]:
msdMessage = {'msdStructure' : {
		'messageIdentifier': 1,
		'control': {
			'automaticActivation': True,
			'testCall' : False,
			'positionCanBeTrusted' :  True,
			'vehicleType' : 1
		},
		'vehicleIdentificationNumber' : {
			'isowmi': 'ECA',
			'isovds': 'LLEXAM',
			'isovisModelyear': 'P',
			'isovisSeqPlant' : 'LE02013'
		},
		'vehiclePropulsionStorageType' : {
			'gasolineTankPresent' : True,
			'dieselTankPresent' : False,
			'compressedNaturalGas' : False,
			'liquidPropaneGas' : False,
			'electricEnergyStorage' : False,
			'hydrogenStorage' : False,
			'otherStorage' : False
		},
		'timestamp' : 1367878452,
		'vehicleLocation' : { 
			'positionLatitude' : 18859320,
			'positionLongitude': 187996428
		},
		'vehicleDirection' :  45,
		'recentVehicleLocationN1': {
			'latitudeDelta' :  0,
			'longitudeDelta': 10
		},
		'recentVehicleLocationN2' : {
			'latitudeDelta' : 0,
			'longitudeDelta' : 30
		},
		'numberOfPassengers' : 2
	}
 }
print(msdMessage)

import binascii
import asn1tools
msd = asn1tools.compile_files('data/msd_v2.asn1', codec='uper',numeric_enums=True)
msdcode = msd.encode('MSDMessage', msdMessage,  check_types=True, check_constraints=True )
print("b'")
print("".join("\\x%02x" % i for i in msdcode))


#msdcode = b'0'
eCallMessage = { 
  'msdVersion':  2,
  'msd' : msdcode
}
print(eCallMessage)

eCallMessageCode = msd.encode('ECallMessage', eCallMessage,check_types=True, check_constraints=True)

print("".join("\\x%02x" % i for i in eCallMessageCode))


import binascii
hexcode = binascii.hexlify(bytearray(eCallMessageCode))
print(hexcode)

print( "Länge des Oct String = " + str(len(eCallMessageCode)) ) 

bytes_as_bits = ''.join("{:08b}\n".format(byte) for byte in eCallMessageCode)

print (bytes_as_bits)

{'msdStructure': {'messageIdentifier': 1, 'control': {'automaticActivation': True, 'testCall': False, 'positionCanBeTrusted': True, 'vehicleType': 1}, 'vehicleIdentificationNumber': {'isowmi': 'ECA', 'isovds': 'LLEXAM', 'isovisModelyear': 'P', 'isovisSeqPlant': 'LE02013'}, 'vehiclePropulsionStorageType': {'gasolineTankPresent': True, 'dieselTankPresent': False, 'compressedNaturalGas': False, 'liquidPropaneGas': False, 'electricEnergyStorage': False, 'hydrogenStorage': False, 'otherStorage': False}, 'timestamp': 1367878452, 'vehicleLocation': {'positionLatitude': 18859320, 'positionLongitude': 187996428}, 'vehicleDirection': 45, 'recentVehicleLocationN1': {'latitudeDelta': 0, 'longitudeDelta': 10}, 'recentVehicleLocationN2': {'latitudeDelta': 0, 'longitudeDelta': 30}, 'numberOfPassengers': 2}}
b'
\x1c\x06\x80\xe3\x0a\x51\x43\x9e\x29\x55\xd4\x38\x00\x80\x04\x34\x0a\x8c\x41\x59\xa4\x08\xfe\x29\xc4\x59\xa4\xc8\x61\x6c\x01\x05\x40\x10\xf0\x10
{'msdVersion': 2, 'msd': bytearray(b'\x1c\x06\x8

Vergleich den Decoder mit 
https://asn1.io/asn1playground/

1. Laden vom MSD V2 ASN-File (Upload ASN.1 File, Durchsuchen :  Desktop\notruf\eCall\eCallTestData\MSD.asn) 
2. Compile (Compiliert zwei Objekte: CurrentVersion, ECallMessage) 
3. Wähle den Decoder (Data: HEX oder Base64, Tpye: ECallMessage, UPER) - > Decode


In [5]:
msd.types

{'CurrentVersion': Integer(CurrentVersion),
 'ECallMessage': Sequence(ECallMessage, [Integer(msdVersion), OctetString(msd)]),
 'MSDMessage': Sequence(MSDMessage, [Sequence(msdStructure, [Integer(messageIdentifier), Sequence(control, [Boolean(automaticActivation), Boolean(testCall), Boolean(positionCanBeTrusted), Enumerated(vehicleType)]), Sequence(vehicleIdentificationNumber, [PrintableString(isowmi), PrintableString(isovds), PrintableString(isovisModelyear), PrintableString(isovisSeqPlant)]), Sequence(vehiclePropulsionStorageType, [Boolean(gasolineTankPresent), Boolean(dieselTankPresent), Boolean(compressedNaturalGas), Boolean(liquidPropaneGas), Boolean(electricEnergyStorage), Boolean(hydrogenStorage), Boolean(otherStorage)]), Integer(timestamp), Sequence(vehicleLocation, [Integer(positionLatitude), Integer(positionLongitude)]), Integer(vehicleDirection), Sequence(recentVehicleLocationN1, [Integer(latitudeDelta), Integer(longitudeDelta)]), Sequence(recentVehicleLocationN2, [Integer(la

Teste UPER Coding von vehiclePropulsionStorageType (Binary with DEFAULTS und Options)

In [5]:
msdMessage = {'vehiclePropulsionStorageType' : {
			'gasolineTankPresent' : True,
			'dieselTankPresent' : False,
			'compressedNaturalGas' : False,
			'liquidPropaneGas' : False,
			'electricEnergyStorage' : False,
			'hydrogenStorage' : False,
			'otherStorage' : False
            }
             }

msd = asn1tools.compile_files('data/msd_v2.asn1', codec='uper',numeric_enums=True)
msdcode = msd.encode('vehiclePropulsionStorageType', msdMessage,  check_types=True, check_constraints=True )

print("b'")
print("".join("\\x%02x" % i for i in msdcode))

EncodeError: Type 'vehiclePropulsionStorageType' not found in types dictionary.

In [38]:
# eCall Message nach  CEN-EN-15722_2015-08
hex_string = '02251C0680E30A51439E2955D43800800437F80A3105669023F8A71166932185B004150043C040'

binary_string = binascii.unhexlify(hex_string)

print (binascii.hexlify(binary_string))

bytes_as_bits = ''.join("{:08b}\n".format(byte) for byte in binary_string)
print (bytes_as_bits)

#result = msd.decode('ECallMessage', bytearray(eCallMessage1))
#print(result)



b'02251c0680e30a51439e2955d43800800437f80a3105669023f8a71166932185b004150043c040'
00000010
00100101
00011100
00000110
10000000
11100011
00001010
01010001
01000011
10011110
00101001
01010101
11010100
00111000
00000000
10000000
00000100
00110111
11111000
00001010
00110001
00000101
01100110
10010000
00100011
11111000
10100111
00010001
01100110
10010011
00100001
10000101
10110000
00000100
00010101
00000000
01000011
11000000
01000000



Mögliche Codecs sind 'ber', 'der', 'gser', 'jer', oer, 'per', 'uper' and 'xer'.

b'
\x1c\x06\x80\xe3\x0a\x51\x43\x9e\x29\x55\xd4\x38\x00\x80\x04\x34\x0a\x8c\x41\x59\xa4\x08\xfe\x29\xc4\x59\xa4\xc8\x61\x6c\x01\x05\x40\x10\xf0\x10


VT Sequenz mit einem Message Identifier der Länge 7-Bit. Danach Boolenns mit Default Werte. 

UPER Codierung: 

- 0-bit für Extension ... (1 bedeutet das die Extenstion vorhanden ist) : 
- Danach 8-bit für die Default Werte der Struktur VT
- Danach Integer-Wert 7 Bit lang (da Wertebereich zwischen 0..127)
- Danach 00 bis ein Byte aufgefüllt ist.
- Total 3Byte fpr die Codierung der Struktur (das macht nun mal bei eCall keinen Sinn), 
ASN.1 ist good old Technology, welche in der EAS für Raummissionen und zur Steuerung von Remote Robos verwendet wird. Hier hat man etwas über das Ziel geschossen. 
- good old Technolgy (1980) aus ISDN CS Zeitalter
- deshalb macht es durchaus Sinn, wenn sich die Telcos darum kümmern (Die verstehen diese Technologie noch!!)
- 

In [13]:
mess =  {
			'gasolineTankPresent' : True,
			'dieselTankPresent' : False,
			'compressedNaturalGas' : True,
			'liquidPropaneGas' : False,
			'electricEnergyStorage' : True,
			'hydrogenStorage' : False,
			'otherStorage' : True,
            'test' : False
}
print (mess)
uper = asn1tools.compile_files('data/optional.asn1', codec='uper' )

gaso = uper.encode('VT', mess, check_types=True, check_constraints=True)

gaso
bytes_as_bits = ''.join("{:08b}\n".format(byte) for byte in gaso)

print("UPER Encoding mit optionalen Felder")
print (bytes_as_bits)
print("".join("\\x%02x" % i for i in gaso))

uper.decode('VT', gaso)

{'gasolineTankPresent': True, 'dieselTankPresent': False, 'compressedNaturalGas': True, 'liquidPropaneGas': False, 'electricEnergyStorage': True, 'hydrogenStorage': False, 'otherStorage': True, 'test': False}
UPER Encoding mit optionalen Felder
01010101
01111000

\x55\x78


{'gasolineTankPresent': True,
 'dieselTankPresent': False,
 'compressedNaturalGas': True,
 'liquidPropaneGas': False,
 'electricEnergyStorage': True,
 'hydrogenStorage': False,
 'otherStorage': True,
 'test': False}

- Beispiel aus CEN-EN-15722_20

NameError: name 'code' is not defined

In [14]:
result = msd.decode('ECallMessage', code)
print(result)

{'msdVersion': 2, 'msd': b'\x1c\x06\x80\xe3\nQC\x9e)U\xd48\x00\x80\x044\n\x8cAY\xa4\x08\xfe)\xc4Y\xa4\xc8al\x01\x05@\x10\xf0\x10'}


In [15]:
print(msd.types['ECallMessage'])

Sequence(ECallMessage, [Integer(msdVersion), OctetString(msd)])


In [16]:
print(msd.types['MSDMessage'])
#print(msd.types['ECallMessage'])

Sequence(MSDMessage, [Sequence(msdStructure, [Integer(messageIdentifier), Sequence(control, [Boolean(automaticActivation), Boolean(testCall), Boolean(positionCanBeTrusted), Enumerated(vehicleType)]), Sequence(vehicleIdentificationNumber, [PrintableString(isowmi), PrintableString(isovds), PrintableString(isovisModelyear), PrintableString(isovisSeqPlant)]), Sequence(vehiclePropulsionStorageType, [Boolean(gasolineTankPresent), Boolean(dieselTankPresent), Boolean(compressedNaturalGas), Boolean(liquidPropaneGas), Boolean(electricEnergyStorage), Boolean(hydrogenStorage), Boolean(otherStorage)]), Integer(timestamp), Sequence(vehicleLocation, [Integer(positionLatitude), Integer(positionLongitude)]), Integer(vehicleDirection), Sequence(recentVehicleLocationN1, [Integer(latitudeDelta), Integer(longitudeDelta)]), Sequence(recentVehicleLocationN2, [Integer(latitudeDelta), Integer(longitudeDelta)]), Integer(numberOfPassengers)]), Sequence(optionalAdditionalData, [OctetString(oid), OctetString(data)

In [17]:
msd.decode('MSDMessage', result['msd'])

{'msdStructure': {'messageIdentifier': 1,
  'control': {'automaticActivation': True,
   'testCall': False,
   'positionCanBeTrusted': True,
   'vehicleType': 'passengerVehicleClassM1'},
  'vehicleIdentificationNumber': {'isowmi': 'ECA',
   'isovds': 'LLEXAM',
   'isovisModelyear': 'P',
   'isovisSeqPlant': 'LE02013'},
  'vehiclePropulsionStorageType': {'gasolineTankPresent': True,
   'dieselTankPresent': False,
   'compressedNaturalGas': False,
   'liquidPropaneGas': False,
   'electricEnergyStorage': False,
   'hydrogenStorage': False,
   'otherStorage': False},
  'timestamp': 1367878452,
  'vehicleLocation': {'positionLatitude': 18859320,
   'positionLongitude': 187996428},
  'vehicleDirection': 45,
  'recentVehicleLocationN1': {'latitudeDelta': 0, 'longitudeDelta': 10},
  'recentVehicleLocationN2': {'latitudeDelta': 0, 'longitudeDelta': 30},
  'numberOfPassengers': 2}}

http://www.auto-vin-decoder.com/
Gültiger VIN-Code : WVWZZZ1JZ3W386752

Gültige PDU Message mit VW Golf / Golf IV Cabrio / Bora / Jetta / Bora Variant - MK4 / R32, 2003,Wolfsburg, Germany, Serial 386752) 

PDU(46octets):015C06B1D71D8208014A00DD0C81871424583ADE68AC52E9BB8413F149C07414FB414F6010180813E82181823230

b'02251c0680e30a51439e2955d43800800437f80a3105669023f8a71166932185b004150043c040'
00000010
00100101
00011100
00000110
10000000
11100011
00001010
01010001
01000011
10011110
00101001
01010101
11010100
00111000
00000000
10000000
00000100
00110111
11111000
00001010
00110001
00000101
01100110
10010000
00100011
11111000
10100111
00010001
01100110
10010011
00100001
10000101
10110000
00000100
00010101
00000000
01000011
11000000
01000000



In [14]:
msd.decode('MSDMessage', result['msd'])

DecodeError: msdStructure: vehicleIdentificationNumber: isowmi: Expected a value in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32], but got 48.

# Nützliche Referenzen zu ASN.1


ASN1 Compiler welcher C++ Code erstellt. Verwendet im ESA Space Programm. Dieser kann auch UPER Coding.  

https://www.thanassis.space/asn1.html#using